[사용자 정의 함수]
- - -
- 함수 기능: 모델 학습 진행 함수
- 함수 이름: training
- 매개변수: 학습 구동 시 필요한 재료(학습을 위한 재료)
    - 모델 인스턴스
    - 학습용 데이터셋: feature, target(학습 전 tensor화 작업 필요)
    - 손실 함수 인스턴스
    - 최적화 인스턴스
    - 학습 횟수: EPOCH
    - 배치 크기: BATCH_SIZE
    - 배치 개수: BATCH_CNT
    - 검증용 데이터셋: feature, target(검증 전 tensor화 작업 필요)
- 함수 결과: 학습 시 에포크 당 손실 값, 성능 지표 값, 검증 시 손실 값, 성능 지표 값 반환
* * *
- 함수 기능: 에포크 단위 모델 학습 진행 함수
- 함수 이름: EPOCH_Training
- 매개변수: 학습 구동 시 필요한 재료(학습을 위한 재료)
    - 모델 인스턴스
    - 학습용 데이터셋: feature, target(학습 전 tensor화 작업 필요)
    - 손실 함수 인스턴스
    - 최적화 인스턴스
    - 배치 크기: BATCH_SIZE
    - 배치 개수: BATCH_CNT
- 함수 결과: 손실 값, 성능 지표 값


In [14]:
#모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizer
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

In [15]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
#에포크 당 학습 진행 후 손실값과 성능지표값 반환 함수
def EPOCH_Training(model,feature,target,lossfunc,optimizer,batch_cnt,batch_size=32,is_class=True):
    
    #에포크에서 배치 크기만큼 데이터셋 추출 후 학습 진행
    loss_total=0
    score_total=0

    for batch in range(batch_cnt):
        #배치 크기만큼 추출 및 tensor화
        start=batch*batch_size
        end=start+batch_size

        #feature,target: DF로 가정
        x_train=torch.FloatTensor(feature[start:end].values).to(device)
        y_train=torch.FloatTensor(target[start:end].values).to(device)

        #학습 진행
        pre_y=model(x_train).to(device)

        #손실 계산
        loss=lossfunc(pre_y,y_train)
        loss_total+=loss

        #점수 추출
        score=F1Score()(pre_y,y_train) if is_class else R2Score()(pre_y,y_train)
        score_total+=score

        #최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    #에포크 당 손실값과 성능지표값 반환 
    

    #테스트 및 검증 함수 사용
    return loss_total/batch_cnt,0

In [17]:
#검증 및 테스트 진행 후 손실값과 성능지표값 반환 함수
def testing(model,feature,target,lossfunc,scorefunc):
    
    #최적화 기능 비활성화 후 데이터셋 추출, 학습 진행
    with torch.no_grad():
        #tensor화
        feature_ts=torch.FloatTensor(feature.values).to(device)
        target_ts=torch.FloatTensor(target.values).to(device)

        #학습 진행
        pre_y=model(feature_ts).to(device)

        #손실 계산
        loss=lossfunc(pre_y,target_ts)

        #점수 추출
        score=scorefunc(pre_y,target_ts)

    #손실값과 성능지표값 반환 
    return loss,score

In [ ]:
#학습 진행

EPOCHS=10
tv_loss={'train':[],'val':[]}
tv_score={'train':[],'val': []}

for epoch in range(EPOCHS):
    #학습 진행
    train_loss,train_score=EPOCH_Training()

    #검증 진행
    val_loss,val_score=testing()

    #에포크 당 학습 및 검증 결과 저장
    tv_loss['train'].append(train_loss)
    tv_score['train'].append(train_score)

    tv_loss['val'].append(val_loss)
    tv_score['val'].append(val_score)

    #에포크 당 학습 및 검증 결과 출력
    print(f'[{epoch}/{EPOCHS}]\n[TRAIN] loss: {train_loss}, score: {train_score}')
    print(f'[VALID] loss: {val_loss}, score: {val_score}')